In [10]:
%%html
<style>
.rise-enabled .cell .input_prompt {
    display: none;
}
</style>
<!--- <style>

# CS 121 Lecture 6 : Turing Machines and Loops

## Utilities
(Ignore at first read)

In [ ]:
from IPython.display import Markdown, Math, display, clear_output
import time

def htmlspace(s):
    return s.replace(" ","&nbsp;")

def printmd(string, color=None):
    colorstr = "<span style='color:{}'>{}</span>".format(color, string) if color else string
    display(Markdown(colorstr))
    
def color(c,s,j=0):
    return "<span style='color:{}'>{}</span>".format(c, s.ljust(j))
    
def bold(s,justify=0):
    return ("**"+s+"**").ljust(justify)
#  return "\x1b[1m"+s.ljust(justify)+"\x1b[0m"

#def underline(s,justify=0):
#    return "\x1b[4m"+s.ljust(justify)+"\x1b[24m"

def red(s,justify=0):
    return color("red",s,justify)  #.replace(" ","&nbsp;")
    #return  "\x1b[31m"+s.ljust(justify)+"\x1b[0m"


def green(s,justify=0):
    return color("green",s,justify) #.replace(" ","&nbsp;")
    # return  "\x1b[32m"+s.ljust(justify)+"\x1b[0m"


def blue(s,justify=0):
    return color("blue",s,justify) #.replace(" ","&nbsp;")
    # return  "\x1b[34m"+s.ljust(justify)+"\x1b[0m"

In [ ]:
from IPython.display import HTML
def mdtable(L,header=True):
    l = max(max([len(str(a)) for a in A]) for A in L)+5
    n = len(L[0])
    res = ""
    sep = "|-".join(["-"*(l+1) for i in range(n)])+"\n"
    for A in L:
        res += " | ".join([bold(str(s),l) if header else str(s).ljust(l) for s in A])+"\n"
        if header:
            res += sep
            header = False
    return Markdown(res)

def htmltable(L,header=True, thicker=100):
    l = max(max([len(str(a)) for a in A]) for A in L)+5
    n = len(L[0])
    res="""<style>
.table-bordered {
    border-collapse: collapse;
}
.table-bordered th, .table-bordered td {
    border: 1px solid black;
    padding: 5px;
    text-align: left !important;
}

.table-bordered th:nth-child(ZZZ), .table-bordered td:nth-child(ZZZ) {
    border-right: 3px solid black;
}
.table-bordered th {
    border-bottom: 3px solid black !important;
}
</style>
<table class="table-bordered">
""".replace("ZZZ",str(thicker))
    res += "<thead>\n" if header else "<tbody>"
    def wrap(i,x):
        if i==0 and header: return f"<th>{x}</th>"
        return f"<td>{x}</td>"
    for i,A in enumerate(L):
        res += "<tr>\n" + "\n".join([wrap(i,x) for x in A]) +"\n</tr>"
        if i==0 and header: res += "\n</thead><tbody>\n"
    res += "\n</tbody><table>"
    return HTML(res)


#L = [["col one", "col 2", "col 3"],[2, 18, 19], [4, 20, 23]]
#mdtable(L)

In [ ]:
def myred(x):
    CRED = '\033[91m'
    CEND = '\033[0m'
    return CRED+x+CEND

def mygreen(x):
    CGREEN= '\033[32m'
    CEND = '\033[0m'
    return CGREEN+x+CEND


In [ ]:
import collections
from collections import defaultdict
import inspect
import re

class QuitExecution(Exception):
    pass

class HaltExecution(Exception):
    pass

def extractstates(transition):
    src = inspect.getsource(transition)
    strings = re.findall( r'(?<!f)"[a-zA-Z\_0-9]*"', src, re.I | re.M)
    strings += re.findall( r"(?<!f)'[a-zA-Z\_0-9]*'", src, re.I | re.M)
    alphabet = [ "▷", "∅","0", "1" ]
    states = [ "START"]
    if src.find("boilerplate(")>=0:
        bpstates = ["OUTPUT0","OUTPUT1","0ANDSTOP","1ANDSTOP"]
    else: 
        bpstates = []
    for s in strings:
        t = s[1:-1]
        if not t: continue
        if len(t)>1:
            if not t in states and not t in bpstates:
                states.append(t)
        else:
            if not t in alphabet and not t in ["L","R","H","S"]:
                alphabet.append(t)
    states += bpstates
    states = [s for s in states if s != 'ANDSTOP']
    return alphabet, states
 

## Actual lecture

__Q:__ Write Turing machine that computes $XOR:\{0,1\}^* \rightarrow \{0,1\}$

__Solution:__ The machine will have a state `EVEN` and `ODD` keeping track of whether so far it has seen an even or odd number of $1$'s. 
When it reaches the end, depending on which state it's in, it will move to either `OUTPUT0` or `OUTPUT1` states, which will cause it to move to the leftmost and output $0$ or $1$ respectivly.


In [11]:
def xortm(state,sym):
    # transition function for XOR Turing Machine
    move = "R"
    if state=="START":  state = "EVEN"
    elif state in ["EVEN","ODD"]:
        if sym=="1":
            # flip state if we see a 1
            state = "EVEN" if state=="ODD" else "ODD"
        elif sym== "∅":
            # at end of tape decide on output based on parity
            state = "OUTPUT1" if state=="ODD" else "OUTPUT0"
            move  = "L"
    # boilerplate
    elif state in ["OUTPUT1","OUTPUT0"]:
        if sym != "▷":
            move, sym  = "L", "∅"
        else:
            move, state = "R" , state[-1]+"ANDSTOP"
    elif state[1:] =="ANDSTOP":
        move, sym = "H", state[0]   
    return state,sym,move

In [13]:
class TuringMachine:
   
    def __init__(self, transition):
        self.alphabet = [ "▷", "∅","0", "1" ] 
        self.transition = transition
        
    def input(self, x):
        self.tape = [ "▷" ] + [str(a) for a in x ] + [ "∅" ]
        self.state = "START"
        self.head = 0 
      
    def run(self):
        while True:
            self.state, self.tape[self.head] , move = self.transition(
                                                           self.state, self.tape[self.head] )
            
            if move=="L": self.head = max(0, self.head-1)
            if move=="R":self.head += 1
            if move=="H": break
            if self.head >= len(self.tape): 
                self.tape.append("∅")
        return self.tape[1]  # more generally, all positions until first "∅"

In [15]:
MXOR = TuringMachine(xortm)
MXOR.input([0,1,1,1,0,0,1,1,0,0])
MXOR.run()

'1'

In [16]:
class TuringMachine:
    """Fancier version with better printing etc..."""
   
    def printtable(self,boiler=False):

        header = ["Curr State", "Curr Sym", "New State", "New Sym", "Move"]
        res = [header]
        bpstates = ["OUTPUT0","OUTPUT1","0ANDSTOP","1ANDSTOP", "ANDSTOP"] if not boiler else []
        skipped = False
        for i, s in enumerate(self.states):
            if s in bpstates: 
                skipped = True
                continue
            first_sym = True
            for a in self.alphabet:
                s_, a_, m_ = self.transition(s,a)
                if first_sym:
                    res.append([f"{i} ({s})", a, s_, a_, m_ ])
                    first_sym = False
                else:
                    res.append(["--", a, s_, a_, m_ ])
                    
        if skipped: res.append(["_boilerplate_","...","...","...","..."])
        
        display(htmltable(res,header=True,thicker=2))
        
        
        
    def _repr_pretty_(self, p, cycle):
        if cycle: return "cycle"
        self.printtable()
        return None

    def __init__(self, transition):
        #self.tape =  collections.defaultdict(lambda :"∅")
        #self.tape[0] = "▷"
        self.print_history = ""
        self.alphabet = [ "▷", "∅","0", "1" ]
        self.transition = transition
        self.head = 0
        self.alphabet , self.states = extractstates(transition)
        self.state = "START"
        self.MAXSTEPS = 60
        self.lenstate = max([len(s) for s in self.states])+2
        self.history = []
        
    def input(self, x):
        self.print_history = ""
        self.tape = [ "▷" ] + [str(a) for a in x ] + [ "∅" ]
        self.state = "START"
        self.head = 0 
    
    def printstate(self, header= True):
        tape_ = [self.tape[i] if i != self.head else myred(self.tape[i]) for i in range(len(self.tape))]
        if header:
            print(mygreen("State".ljust(self.lenstate))+ " " +mygreen("Tape"))
        print(self.state.ljust(self.lenstate) + " "+ "".join(tape_))
        return None
    
    def config(self):
        n = len(self.states)
        k = self.states.index(self.state)
        l = len(str(n-1))
        tlen = len(self.tape)
        for i in range(tlen-1,0,-1):
            if self.tape[i] != "∅":
                break
        tlen = i+1
        alph = r"\{" + ", ".join(self.alphabet) +r"\}"
        states = f"([{n}]" + r" \cup \{ \cdot \})"
        display(Math(r"\text{configuration} \in \left(" + alph + r"  \times " + states +r"\right)^*" ))
        tape_ = [self.tape[i].ljust(l) if i != self.head else myred(self.tape[i].ljust(l)) for i in range(tlen)]
        states_ = ["⋅".ljust(l) if i != self.head else myred(str(k).ljust(l))for i in range(tlen)]
        print("".join(tape_))
        print("".join(states_))
        print(f"\nState {k} =  {self.state}")
        
        
    
    def next(self):
        self.history.append((self.state, self.head, list(self.tape)))
        self.state,self.tape[self.head],move = self.transition(self.state, self.tape[self.head])
        if not self.state in self.states:
            self.states.append(self.state)
            
        if move=="L":
            self.head = max(0, self.head-1)
        if move =="R":
            self.head += 1
            if self.head == len(self.tape):
                self.tape.append("∅")
        if move =="H":
            raise HaltExecution("Halted")
    
    def prev(self):
        if not self.history:
            raise Exception("Can't go back - history is empty")
        self.state,self.head, self.tape = self.history.pop()
        
    def run(self,iterate = False, maxsteps = 0):
        if not maxsteps:
            maxsteps = self.MAXSTEPS
        t = 0
        noprinting = 0
        quit_cmd = False
        if iterate:
            print("q(uit), n(ext),p(rev),c(lear),r(un),s(kip) XX:")
        try:
            while True:
                if noprinting>0:
                    noprinting -= 1
                else:
                    if iterate: 
                        ... # self.reprint()
                    else:
                        time.sleep(0.5)
                    self.printstate(header = t==0)
                if iterate and noprinting<=0:
                    cmd = input("")
                    #CURSOR_UP_ONE = '\x1b[1A'
                    #ERASE_LINE = '\x1b[2K'
                    #print(CURSOR_UP_ONE + ERASE_LINE + CURSOR_UP_ONE)
                    
                    c = cmd[0] if cmd else "n"
                    if c=="c":
                        clear_output()
                    elif c=="r":
                        iterate = False
                    elif c=="s":
                        noprinting = int(cmd[1:].strip())
                        print("...")
                    elif c=="p":
                        self.prev()
                        t -= 1
                        continue
                    elif c=="q":
                        raise QuitExecution("User quit")
                self.next()
                if t >= maxsteps:
                    raise Exception("Too many steps")
                t += 1
               
        except HaltExecution as e:
            msg = str(e)
            self.printstate(header = iterate)
            y = ""
            i = 1
            while self.tape[i] != "∅":
                y += self.tape[i]
                i += 1
            return y
        except  QuitExecution as e:
            print(str(e))
            

            

In [17]:
MXOR = TuringMachine(xortm)
MXOR.input([1,0,0,1,1]) 
MXOR.run()

State     Tape
START     ▷10011∅
EVEN      ▷10011∅
ODD       ▷10011∅
ODD       ▷10011∅
ODD       ▷10011∅
EVEN      ▷10011∅
ODD       ▷10011∅
OUTPUT1   ▷10011∅
OUTPUT1   ▷1001∅∅
OUTPUT1   ▷100∅∅∅
OUTPUT1   ▷10∅∅∅∅
OUTPUT1   ▷1∅∅∅∅∅
OUTPUT1   ▷∅∅∅∅∅∅
1ANDSTOP  ▷∅∅∅∅∅∅
1ANDSTOP  ▷1∅∅∅∅∅


'1'

In [18]:
MXOR.input([1,0,0,1]) 
MXOR.run()

State     Tape
START     ▷1001∅
EVEN      ▷1001∅
ODD       ▷1001∅
ODD       ▷1001∅
ODD       ▷1001∅
EVEN      ▷1001∅
OUTPUT0   ▷1001∅
OUTPUT0   ▷100∅∅
OUTPUT0   ▷10∅∅∅
OUTPUT0   ▷1∅∅∅∅
OUTPUT0   ▷∅∅∅∅∅
0ANDSTOP  ▷∅∅∅∅∅
0ANDSTOP  ▷0∅∅∅∅


'0'

In [21]:
MXOR.printtable(True)

Curr State,Curr Sym,New State,New Sym,Move
0 (START),▷,EVEN,▷,R
--,∅,EVEN,∅,R
--,0,EVEN,0,R
--,1,EVEN,1,R
1 (EVEN),▷,EVEN,▷,R
--,∅,OUTPUT0,∅,L
--,0,EVEN,0,R
--,1,ODD,1,R
2 (ODD),▷,ODD,▷,R
--,∅,OUTPUT1,∅,L


In [22]:
MXOR.input([1,0,1])
MXOR.run(True)

q(uit), n(ext),p(rev),c(lear),r(un),s(kip) XX:
State     Tape
START     ▷101∅

EVEN      ▷101∅

ODD       ▷101∅

ODD       ▷101∅

EVEN      ▷101∅
q
User quit


In [ ]:
# Confugation of a Turing machine
MXOR.input([1,0,1])
MXOR.next()
MXOR.next()
MXOR.next()
MXOR.config()

### Factor out boilerplate

(Can skip)

In [ ]:
def xortm(state,sym):
    move = "R"
    if state=="START":
        state = "EVEN"
    elif state in ["EVEN","ODD"]:
        if sym== "∅":
            state = "OUTPUT1" if state=="ODD" else "OUTPUT0"
            move  = "L"
        elif sym=="1":
            state = "EVEN" if state=="ODD" else "ODD"
    else:
        state,sym,move = boilerplate(state,sym)
    return state,sym,move

In [26]:
def boilerplate(state,sym):
    move = "H"
    if state in ["OUTPUT1","OUTPUT0"]:
        if sym != "▷":
            move = "L"
            sym= "∅"
        else:
            move = "R"
            state = state[-1]+"ANDSTOP"
    elif state[1:] =="ANDSTOP":
        move = "H"
        sym = state[0]
    return state,sym,move


In [ ]:
MXOR = TuringMachine(xortm)
MXOR.printtable(False)

In [ ]:
MXOR.input("1011")
MXOR.printstate()

In [ ]:
MXOR.run(True)

## Palindrome

__Q:__ Write a TM $M$ such that for every $x\in \{0,1\}^n$, if $M$'s tape is initialized with $\triangleright x_0 x_1 \cdots x_{n-1} \cdots$ then when it halts the tape is $\triangleright PAL(x) \cdots$ where $PAL(x)=1$ iff $x_i=x_{n-i}$ for every $i\in [n]$.

In [27]:
def palindrome(state,sym):
    if sym== "▷" and state in ["START","SCANLEFT"]:
        state = "LEFTMOST"
        move = "R"
    elif state == "LEFTMOST":
        if sym == "∅": state, move = "OUTPUT1","S"
        else:
            state = "LOOK0_SAW0" if sym=="0" else "LOOK1_SAW1"
            sym,move  = "∅","R"
    elif state in [ "LOOK0_SAW0", "LOOK0_SAW1", "LOOK1_SAW0", "LOOK1_SAW1" ]:
        if sym == "∅":
            state = "OUTPUT0" if state[4] != state[9] else "ERASE"
            move  = "L"
        else: state, move = state[:-1]+sym, "R"
    elif state == "ERASE":
        if sym == "∅": state,move = "LEFTMOST","R"
        else: state,sym,move = "SCANLEFT", "∅", "L"
    elif state == "SCANLEFT":
        if sym == "∅": state,move = "LEFTMOST","R"
        else: move = "L"
    else:
        state,sym,move = boilerplate(state,sym)
    return state,sym,move

In [28]:
PalM = TuringMachine(palindrome)
PalM

Curr State,Curr Sym,New State,New Sym,Move
0 (START),▷,LEFTMOST,▷,R
--,∅,START,∅,H
--,0,START,0,H
--,1,START,1,H
1 (SCANLEFT),▷,LEFTMOST,▷,R
--,∅,LEFTMOST,∅,R
--,0,SCANLEFT,0,L
--,1,SCANLEFT,1,L
2 (LEFTMOST),▷,LOOK1_SAW1,∅,R
--,∅,OUTPUT1,∅,S


In [30]:
PalM.input("110")
PalM.run()

State        Tape
START        ▷110∅
LEFTMOST     ▷110∅
LOOK1_SAW1   ▷∅10∅
LOOK1_SAW1   ▷∅10∅
LOOK1_SAW0   ▷∅10∅
OUTPUT0      ▷∅10∅
OUTPUT0      ▷∅1∅∅
OUTPUT0      ▷∅∅∅∅
OUTPUT0      ▷∅∅∅∅
0ANDSTOP     ▷∅∅∅∅
0ANDSTOP     ▷0∅∅∅


'0'

## Turing Machines don't have to halt!!

In [31]:
def nonhalting(state,sym):
    return state,sym,"R"

NH = TuringMachine(nonhalting)
NH

Curr State,Curr Sym,New State,New Sym,Move
0 (START),▷,START,▷,R
--,∅,START,∅,R
--,0,START,0,R
--,1,START,1,R


In [32]:
NH.input("00111011")
NH.printstate()

State   Tape
START   ▷00111011∅


In [33]:
NH.run(True)

q(uit), n(ext),p(rev),c(lear),r(un),s(kip) XX:
State   Tape
START   ▷00111011∅

START   ▷00111011∅

START   ▷00111011∅

START   ▷00111011∅

START   ▷00111011∅

START   ▷00111011∅

START   ▷00111011∅

START   ▷00111011∅

START   ▷00111011∅

START   ▷00111011∅

START   ▷00111011∅∅

START   ▷00111011∅∅∅

START   ▷00111011∅∅∅∅

START   ▷00111011∅∅∅∅∅

START   ▷00111011∅∅∅∅∅∅

START   ▷00111011∅∅∅∅∅∅∅

START   ▷00111011∅∅∅∅∅∅∅∅

START   ▷00111011∅∅∅∅∅∅∅∅∅

START   ▷00111011∅∅∅∅∅∅∅∅∅∅

START   ▷00111011∅∅∅∅∅∅∅∅∅∅∅

START   ▷00111011∅∅∅∅∅∅∅∅∅∅∅∅

START   ▷00111011∅∅∅∅∅∅∅∅∅∅∅∅∅

START   ▷00111011∅∅∅∅∅∅∅∅∅∅∅∅∅∅

START   ▷00111011∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅

START   ▷00111011∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅

START   ▷00111011∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅

START   ▷00111011∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅

START   ▷00111011∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅

START   ▷00111011∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅

START   ▷00111011∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅

START   ▷00111011∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅

START   ▷00111011∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅

START   ▷00111011∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅∅

STAR

__Def:__ If $M$ is Turing Machine and $x\in \{0,1\}^*$ then $M(x)$ denotes the contents of tape of $M$ from second position  until  first empty spot. 

If $M(x)$ doesn't halt we denote $M(x) = \bot$.

__Super important def:__ A TM $M$ _computes_ $F:\{0,1\}^* \rightarrow \{0,1\}^*$  if for every $x\in \{0,1\}^*$, $M(x) = F(x)$.

$F$ is _computable_ if there is a TM that computes it

Back to Powerpoint